In [1]:
import cv2
import pickle
import random
import numpy as np
import pandas as pd
from keras.models import Sequential
from matplotlib import pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import confusion_matrix, precision_score
print("Import Libraries")

Import Libraries


In [3]:
def CNN2D(nrows, ncols):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(nrows, ncols, 1)))
    model.add(MaxPooling2D(pool_size=(2, 3)))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 3)))
    model.add(Conv2D(filters=48, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 3)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    print('---2D CNN for Image Size '+str(nrows)+'x'+str(ncols)+'--Developed by Rohit-----')
    print(model.summary())
    return model
print("Define the CNN Model")

Define the CNN Model


In [5]:
# Create dataset array
data = np.zeros((2300, 60, 150))

# Create label array
datalabel = np.zeros((2300, 2))

# Open images and store values in the dataset array for 'open' images
for i in range(0, 1150):
    try:
        I = cv2.imread(f"C:/Users/ROHIT KHETO/Desktop/4th_year/mergeOpenEyes/open{i}.jpg")
        if I is None:
            raise FileNotFoundError(f"open{i} failed to load.")  # Include image number
        
        I = cv2.resize(I, (150, 60))  # Resize the image to (height, width) = (60, 150)
        I = I / 255.0  # Normalize the image
        data[i, :, :] = I[:, :, 0]  # Store the first channel
        datalabel[i, 0] = 1  # Labeling open image
    
    except Exception as e:
        print(e)

# Load 'closed' images
for i in range(0, 1150):
    try:
        I = cv2.imread(f"C:/Users/ROHIT KHETO/Desktop/4th_year/mergeClosedEyes/close{i}.jpg")
        if I is None:
            raise FileNotFoundError(f"close{i} failed to load.")  # Include image number
        
        I = cv2.resize(I, (150, 60))
        I = I / 255.0
        data[i + 1150, :, :] = I[:, :, 0]  # Store the first channel
        datalabel[i + 1150, 1] = 1  # Labeling closed image
    
    except Exception as e:
        print(e)

print("Prepare Dataset and Labels")

Prepare Dataset and Labels


In [7]:
index = np.arange(2300)
random.shuffle(index)
Sdata = np.zeros((2300, 60, 150))
Sdatalabel = np.zeros((2300, 2))

for i in range(0, 2300):
    Sdata[i, :, :] = data[index[i], :, :]
    Sdatalabel[i, :] = datalabel[index[i], :]

print("Data shuffling completed")

Data shuffling completed


In [9]:
nFold = 5
noSample = Sdata.shape[0]
nrows = Sdata.shape[1]
ncols = Sdata.shape[2]
packet = int(noSample / nFold)
print("Set Training Parameter")

Set Training Parameter


In [11]:
trainData = np.zeros(((nFold - 1) * packet, nrows, ncols, 1))
trainClass = np.zeros(((nFold - 1) * packet, 2))
validData = np.zeros((packet, nrows, ncols, 1))
validClass = np.zeros((packet, 2))

final_accuracy = np.zeros((nFold + 1, 1))

for i in range(0, nFold):
    print('=============================================================')
    print('===================== Fold ', str(i), ' is running ...............................')
    print('=============================================================')

    # Preparing training and validation data
    trainData[0:i * packet, :, :, 0] = Sdata[0:i * packet, :, :]
    trainClass[0:i * packet] = Sdatalabel[0:i * packet, :]

    validData[0:packet, :, :, 0] = Sdata[i * packet:((i + 1) * packet), :, :]
    validClass[0:packet] = Sdatalabel[i * packet:((i + 1) * packet), :]

    trainData[i * packet:(nFold - 1) * packet, :, :, 0] = Sdata[(i + 1) * packet:noSample, :, :]
    trainClass[i * packet:(nFold - 1) * packet] = Sdatalabel[(i + 1) * packet:noSample, :]

    model = CNN2D(nrows, ncols)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    results = model.fit(trainData, trainClass, batch_size=32, epochs=100, validation_data=(validData, validClass))

    # Save the model
    model.save(f'C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/Fold_{str(i)}.h5')
    print('Model Saved')

    # Store accuracy results
    acc = np.array(results.history['accuracy'])
    val_acc = np.array(results.history['val_accuracy'])
    noEpoc = acc.shape[0]
    accuracy = np.zeros((noEpoc, 2))
    accuracy[:, 0] = acc
    accuracy[:, 1] = val_acc

    pd.DataFrame(accuracy).to_csv(f'C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/AccuracyFold_{str(i)}.csv')

    # Plot and save accuracy
    data = pd.read_csv(f'C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/AccuracyFold_{str(i)}.csv')
    data = np.array(data)
    n, c = data.shape
    epocRange = np.arange(1, n + 1)
    accData = data[:, 1]
    val_accData = data[:, 2]
    plt.plot(epocRange, accData)
    plt.plot(epocRange, val_accData, '--')
    plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Epoch vs. Accuracy')
    plt.savefig(f'C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/AccuracyFold_{str(i)}.png')  # Save the Image file
    plt.close()

    final_accuracy[i, 0] = val_accData[noEpoc - 1]
    final_accuracy[nFold, 0] = np.mean(final_accuracy[0:nFold, 0])
pd.DataFrame(final_accuracy).to_csv('C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/Final_Accuracy_Foldwise.csv') # Save the final accuracy
print("5 Fold-Cross-Validation Training")

===================== Fold  0  is running ...............................


C:\Users\ROHIT KHETO\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


---2D CNN for Image Size 60x150--Developed by Rohit-----


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 60, 150, 16)         │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 30, 50, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 30, 50, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 15, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 15, 16, 48)          │          13,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 7, 5, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 7, 5, 64)            │          27,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 3, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 384)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          12,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

 Total params: 58,770 (229.57 KB)

 Trainable params: 58,770 (229.57 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.5410 - loss: 0.6852 - val_accuracy: 0.7196 - val_loss: 0.5851
Epoch 2/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.7020 - loss: 0.5731 - val_accuracy: 0.7804 - val_loss: 0.4647
Epoch 3/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.8271 - loss: 0.4029 - val_accuracy: 0.8891 - val_loss: 0.2856
Epoch 4/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8944 - loss: 0.2732 - val_accuracy: 0.9087 - val_loss: 0.2169
Epoch 5/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9289 - loss: 0.1981 - val_accuracy: 0.9435 - val_loss: 0.1603
Epoch 6/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9370 - loss: 0.1569 - val_accuracy: 0.9370 - val_loss: 0.1660
Epoch 7/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9529 - loss: 0.1364 - val_accuracy: 0.9522 - val_loss: 0.1179
Epoch 8/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.9603 - loss: 0.1123 - val_accurac

Model Saved
5 Fold-Cross-Validation Training


In [13]:
# Specify the path to save the model
model_save_path = "C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/model.pkl"

# Save the model
with open(model_save_path, 'wb') as file:
    pickle.dump(model, file) 
    print(f"Model saved successfully at: {model_save_path}")

Model saved successfully at: C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/model.pkl
